# Homework 3 – Deployment

## Tasks

![Iris flower](./data/iris.png)

1. Import Iris dataset from `./data/iris.csv`.
* Create an endpoint `/train` which expects `n_estimators` and `max_depth` attributes of a Random Forest Classifier, trains a model using a Cross Validation, and returns the F1-Score, Precision, Recall and Accuracy.
* Create an endpoint `/predict` which expects the four input features (sepal length, sepal width, petal length, petal width) and predicts as well as retuns the iris class using the model from above.

### Import the dataset

In [1]:
import pandas as pd

df = pd.read_csv('./data/iris.csv')

In [2]:
df.shape

(150, 5)

In [3]:
df.sample()

,sepal length,sepal width,petal length,petal width,target
138,6.0,3.0,4.8,1.8,Iris-virginica


In [4]:
df.head()

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Prepare Data for Decision Tree Classifier

In [5]:
#convert category (strings) into integers
df['target'] = df['target'].astype('category')
iris_types = df['target'].unique()
df['target'] = df['target'].cat.codes
iris_types

[Iris-setosa, Iris-versicolor, Iris-virginica]
Categories (3, object): [Iris-setosa, Iris-versicolor, Iris-virginica]

In [6]:
#split up between features and target
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
print(X.shape,y.shape)

(150, 4) (150,)


### Create Endpoint */train*

In [7]:
from flask import jsonify,Flask,request
app =Flask(__name__)

In [8]:
@app.route('/train')
def train():
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import metrics
    from sklearn.model_selection import KFold
    from sklearn.externals import joblib
    
    #get input for n_estimators and max_depth
    n_estimators = int(request.args.get('n_estimators'))
    max_depth = int(request.args.get('max_depth'))
        
    #initialize Classifier
    rfc = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, random_state = 0)
    
    #Train the Classifier via Crossvalidation
    kf = KFold(n_splits=5,  random_state=None) 
    for train_index, test_index in kf.split(X):      
        tmp_X_train, tmp_X_test = X.iloc[train_index], X.iloc[test_index] 
        tmp_y_train, tmp_y_test = y.iloc[train_index], y.iloc[test_index]
        
        rfc.fit(tmp_X_train,tmp_y_train)
        
    
    #Make prediction with trained Model
    y_pred = rfc.predict(X)
    #make ML model accessible for other methods    
    joblib.dump(rfc,'ML_model.pkl')
    
    return jsonify({'F1-Score': metrics.f1_score(y, y_pred, average = 'weighted'),
                    'Precision': metrics.precision_score(y,y_pred, average = 'weighted'),
                    'Recall': metrics.recall_score(y,y_pred, average = 'weighted'),                             
                    'Accuracy': rfc.score(X,y)
                   })

### Create Endpoint */predict*

In [9]:
@app.route('/predict')
def predict():
    from sklearn.externals import joblib
    
    #get input data
    s_l = float(request.args.get('sepal_length'))
    s_w = float(request.args.get('sepal_width'))
    p_l = float(request.args.get('petal_length'))
    p_w = float(request.args.get('petal_width'))
    
    #create DataFrame of input data
    iris = pd.DataFrame({'sepal_length': [s_l],
                        'sepal_width': [s_w],
                        'petal_length': [p_l],
                        'petal_width': [p_w]})
    
    #get trained model    
    rfc = joblib.load('ML_model.pkl')
    
    #predict with trained model
    output = int(rfc.predict(iris))
    
    #return type of iris
    return jsonify({'Iris type': iris_types[output]})


In [ ]:
app.run(debug=True, use_reloader=False, host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2018 10:32:20] "GET /train?n_estimators=3&max_depth=2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2018 10:32:27] "GET /train?n_estimators=1&max_depth=2 HTTP/1.1" 200 -
C:\Users\Tim Bo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Tim Bo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
127.0.0.1 - - [19/Nov/2018 10:32:30] "GET /train?n_estimators=1&max_depth=1 HTTP/1.1" 200 -
C:\Users\Tim Bo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  '